In [1]:
from fyers_apiv3 import fyersModel
from fyers_apiv3.FyersWebsocket import data_ws

import pandas as pd
import numpy as np
import math

import datetime as dt
from datetime import date, timedelta,datetime
import time 

from fyers_apiv3.FyersWebsocket import data_ws
import csv

import matplotlib.pyplot as plt
import mplfinance as mpf

import scipy

from  time import sleep
import threading
import os
import pyotp
import requests
import json
import pytz
from urllib.parse import parse_qs,urlparse
import base64

from IPython.display import clear_output

import warnings
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [2]:
redirect_uri = "https://trade.fyers.in/api-login/redirect-uri/index.html"
client_id='HTHFWXFS49-100'
secret_key = 'JOQVVTY9UJ'
FY_ID = "YB00632"  
TOTP_KEY = "3Y7PAJSPNHXEADGRDD726MLHJW43AJGH"  
PIN = "1964" 

grant_type = "authorization_code"                  
response_type = "code"                         
state = "sample"                                  

appSession = fyersModel.SessionModel(client_id = client_id, redirect_uri = redirect_uri,response_type=response_type,state=state,secret_key=secret_key,grant_type=grant_type)


generateTokenUrl = appSession.generate_authcode()
generateTokenUrl


def getEncodedString(string):
    string = str(string)
    base64_bytes = base64.b64encode(string.encode("ascii"))
    return base64_bytes.decode("ascii")
  



URL_SEND_LOGIN_OTP="https://api-t2.fyers.in/vagator/v2/send_login_otp_v2"
res = requests.post(url=URL_SEND_LOGIN_OTP, json={"fy_id":getEncodedString(FY_ID),"app_id":"2"}).json()   

if datetime.now().second % 30 > 27 : sleep(5)
URL_VERIFY_OTP="https://api-t2.fyers.in/vagator/v2/verify_otp"
res2 = requests.post(url=URL_VERIFY_OTP, json= {"request_key":res["request_key"],"otp":pyotp.TOTP(TOTP_KEY).now()}).json()  


ses = requests.Session()
URL_VERIFY_OTP2="https://api-t2.fyers.in/vagator/v2/verify_pin_v2"
payload2 = {"request_key": res2["request_key"],"identity_type":"pin","identifier":getEncodedString(PIN)}
res3 = ses.post(url=URL_VERIFY_OTP2, json= payload2).json()  


ses.headers.update({
    'authorization': f"Bearer {res3['data']['access_token']}"
})


TOKENURL="https://api-t1.fyers.in/api/v3/token"
payload3 = {"fyers_id":FY_ID,
           "app_id":client_id[:-4],
           "redirect_uri":redirect_uri,
           "appType":"100","code_challenge":"",
           "state":"None","scope":"","nonce":"","response_type":"code","create_cookie":True}

res3 = ses.post(url=TOKENURL, json= payload3).json()  


url = res3['Url']
parsed = urlparse(url)
auth_code = parse_qs(parsed.query)['auth_code'][0]


grant_type = "authorization_code" 

response_type = "code"  

session = fyersModel.SessionModel(
    client_id=client_id,
    secret_key=secret_key, 
    redirect_uri=redirect_uri, 
    response_type=response_type, 
    grant_type=grant_type
)

session.set_token(auth_code)

response = session.generate_token()

access_token = response['access_token']

fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path=os.getcwd())
fyers.get_profile()

{'s': 'ok',
 'code': 200,
 'message': '',
 'data': {'fy_id': 'YB00632',
  'name': 'BABAN MARUTI PAWAR',
  'image': None,
  'display_name': None,
  'pin_change_date': '21-12-2023 21:01:26',
  'email_id': 'babanpawar8975@gmail.com',
  'pwd_change_date': None,
  'PAN': '---------',
  'mobile_number': '8975963982',
  'totp': True,
  'pwd_to_expire': 90}}

In [3]:
sym = 'NSE:NIFTYBANK-INDEX'

In [4]:
csv_file_path = 'prev_open_date.csv'

with open(csv_file_path, 'r') as file:
    csv_reader = csv.reader(file)
    
    for row in csv_reader:
        cell_value = row[0]

start_date = datetime.strptime(cell_value, '%Y-%m-%d')
start_date = start_date.date()
end_date = date.today()
print("Start Date:", start_date)
print("End Date:", end_date)

Start Date: 2024-01-04
End Date: 2024-01-08


In [5]:
csv_file_path = 'monthly_expiry.csv'

with open(csv_file_path, 'r') as file:
    csv_reader = csv.reader(file)
    
    for row in csv_reader:
        cell_value = row[0]

monthly_exp_date = datetime.strptime(cell_value, '%Y-%m-%d')
monthly_exp_date=monthly_exp_date.date()
tdate = date.today()
print("Monthly Expiry:", monthly_exp_date)

Monthly Expiry: 2024-01-24


In [6]:
csv_file_path = 'today_lvls.csv'

# Create an empty list to store values from column N
lvl = []

# Open the CSV file
with open(csv_file_path, 'r') as file:
    # Create a CSV reader object using DictReader
    csv_reader = csv.DictReader(file)

    # Iterate over the rows in the CSV file
    for row in csv_reader:
        # Assuming 'N' is the header for the column
        # Append the value from column 'N' to the list
        lvl.append(float(row['BN']))

# Now, 'column_n_values' contains all the values from column 'N'
print("lvl:", lvl)

lvl: [48350.5, 48282.05, 48066.95, 47864.45, 47734.75, 47734.75, 47734.75]


In [7]:
historydata = pd.DataFrame()

In [8]:
def gethistorydata(st,ed,res):
    data = {
            "symbol":str(sym),
            "resolution":str(res),
            "date_format":"1",
            "range_from":st,
            "range_to":ed,
            "cont_flag":"0"
            }
    repo = fyers.history(data=data)
#     print(repo)
    data = pd.DataFrame.from_dict(repo['candles'])
    #global cols
    cols = ['datetime', 'open', 'high','low','close', 'volume']
    data.columns = cols
    
    data['datetime'] = pd.to_datetime(data['datetime'],unit="s")
    data['datetime'] = data['datetime'].dt.tz_localize('utc').dt.tz_convert('Asia/Kolkata')
    global historydata
#     data = data.set_index('datetime')
    historydata = data
    historydata['datetime'] = pd.to_datetime(historydata['datetime'])

    historydata['date'] = historydata['datetime'].dt.date
    historydata['time'] = historydata['datetime'].dt.time

    historydata.drop(columns=['datetime'], inplace=True)
    historydata.drop(columns=['volume'], inplace=True)
    
    #append(historydata,data)],axis=0)
    #historydata.to_csv("75op.csv")
    #print(historydata)

In [9]:
def pointchecker(pt,rg):
    point_value = pt
#     rg = 30
    for level in lvl:
        if abs(point_value - level) <= rg:
            return True
    else:
            return False

In [10]:
def plot_tolerance_range(level, arr, tolerance=30, color='lightblue', alpha=0.3):
    lower_bound = level - tolerance
    upper_bound = level + tolerance
    plt.fill_between(range(len(arr)), lower_bound, upper_bound, color=color, alpha=alpha,label='level range')
    plt.axhline(y=level, color='blue', linestyle='-', label=f'Level {level}')

In [11]:
def plot_candles():
    mpf.plot(historydata, type='candle', style='yahoo', title='Candlestick Chart', ylabel='Price', hlines=lvl)

In [12]:
def count_candles(df, start_candle_value_high, start_candle_value_low, end_candle_value_high, end_candle_value_low, candle_value_column1="high", candle_value_column2="low"):
    counts = 0
    
    for index, row in df.iterrows():
        candle_high = row[candle_value_column1]
        candle_low = row[candle_value_column2]
        
        if(start_candle_value_high > candle_high and start_candle_value_low < candle_low):
            counts += 1
        else:
            count=0
            break
            
    return counts

In [13]:
def find_closest_level(point, levels, direction='upward'):
    closest_level = None
    min_distance = float('inf')

    for level in levels:
        distance = abs(level - point)

        if direction == 'upward' and level > point:
            if distance < min_distance:
                min_distance = distance
                closest_level = level
        elif direction == 'downward' and level < point:
            if distance < min_distance:
                min_distance = distance
                closest_level = level

    return closest_level

In [14]:
def count_back_to_back_within_range(dataframe, column_name, target_value, margin):
    if len(dataframe) < 2:
        return 0

    # Iterate over rows to check back-to-back condition
    count_of_back_to_back = 0
    for i in range(1, len(dataframe)):
        current_value = dataframe[column_name].iloc[i]
        previous_value = dataframe[column_name].iloc[i - 1]

        # Check if current and previous values are within the specified range
        if (current_value >= target_value - margin) and (current_value <= target_value + margin) and \
           (previous_value >= target_value - margin) and (previous_value <= target_value + margin):
            count_of_back_to_back += 1

    return count_of_back_to_back

In [15]:
global historydata
itr=0
secitr=0

sl=0
target=0
rrr = 0

ready=0
count=0
status =0
tobs = 0
secitr=0
point1=0
point2=0
c=0
c1=0

sp=''
strike=""
vrti=0

In [16]:
current_time = time.localtime(time.time())
day_of_week = current_time.tm_wday

In [17]:
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
today = days[day_of_week]
print("Day of the week:", today)

Day of the week: Monday


In [18]:
current_time = time.localtime(time.time())

# Extract year, month, and day
year = current_time.tm_year
month = current_time.tm_mon
day = current_time.tm_mday

# Print today's date
print("Today's date:", f"{year}-{month:02d}-{day:02d}")

Today's date: 2024-01-08


In [19]:
def onmessage(message):
#     clear_output(wait=True)
    print(message,"\ttime: ",time.localtime().tm_hour,time.localtime().tm_min,time.localtime().tm_sec)
    time.sleep(1)

    t = time.localtime()
    cmin = time.strftime("%M", t)
    csec = time.strftime("%S", t)  
    
    global historydata,itr,secitr,sl,target,rrr,ready,count,status,tobs,secitr,point1,point2,c,c1,sp,strike,vrti

    for sigma in [0.001, 0.002, 0.0005]:
        if(ready==1 and itr==1):
            c1= count_candles(historydata, historydata['high'].iloc[0],historydata['low'].iloc[0], historydata['high'].iloc[-2], historydata['low'].iloc[-2])
            if (c == c1 and c!=0 and c1!=0 and ((message['ltp'] <= (point1 + 5) and message['ltp'] >= (point1 - 5)) or (message['ltp'] <= (point2 + 5) and message['ltp'] >= (point2 - 5)))):
                if(vrti==0):
                    entryprice = message['ltp']
                    if(today=="Wednesday" or tdate==monthly_exp_date):
                        sp =(int(math.floor(entryprice / 100.0)) * 100)-100
                    else:
                        sp =(int(math.floor(entryprice / 100.0)) * 100)

                    strike = expiry + str(sp) + "PE"
                    edata = {
                    "symbol": str(strike),
                    "qty":15,
                    "type":2,
                    "side":1,
                    "productType":"INTRADAY",
                    "limitPrice":0,
                    "stopPrice":0,
                    "validity":"DAY",
                    "disclosedQty":0,
                    "offlineOrder":False,
                    }
                    eresponse = fyers.place_order(data=edata)
                    print(eresponse)

                    print("ENTRY MADE FOR ",sigma,"ON ",entryprice)
                    entryhour = time.localtime(time.time()).tm_hour
                    entrymin =  time.localtime(time.time()).tm_min
                    entrydate = date.today()
                    count = count+1
                    status = 1
                    start_time = time.time()     
                else:
                    entryprice = message['ltp']
                    if(today=="Wednesday" or tdate==monthly_exp_date):
                        sp =(int(math.floor(entryprice / 100.0)) * 100)-100
                    else:
                        sp =(int(math.floor(entryprice / 100.0)) * 100)

                    strike = expiry + str(sp) + "CE"
                    edata = {
                    "symbol": str(strike),
                    "qty":15,
                    "type":2,
                    "side":1,
                    "productType":"INTRADAY",
                    "limitPrice":0,
                    "stopPrice":0,
                    "validity":"DAY",
                    "disclosedQty":0,
                    "offlineOrder":False,
                    }
                    eresponse = fyers.place_order(data=edata)
                    print(eresponse)

                    print("ENTRY MADE FOR ",sigma,"ON ",entryprice)
                    entryhour = time.localtime(time.time()).tm_hour
                    entrymin =  time.localtime(time.time()).tm_min
                    entrydate = date.today()
                    count = count+1
                    status = 1
                    start_time = time.time()                       
                

        if (int(cmin) % 5 == 0 and int(csec)==1):
            gethistorydata(start_date,end_date,5)  
            plot_candles()

            point1 = historydata['high'].iloc[0]
            point2 = historydata['low'].iloc[0]

            if(historydata['close'].iloc[-1]>point1 and historydata['close'].iloc[-2]<=point1):
                c= count_candles(historydata, historydata['high'].iloc[0],historydata['low'].iloc[0], historydata['high'].iloc[-2], historydata['low'].iloc[-2])
                if(c>2):
                    for i in lvl:
                        if(i<point1 and i>point1-60):
                            sl = i

                        else:
                            sl = point1-60

                    target = find_closest_level(point1,lvl) -30
                    rrr = (abs(target-point1))/(point1-sl)
                    ready=1
                    print("\nCLOSING BHETLI !!! ON ",sigma)

            elif(historydata['close'].iloc[-1]<point2 and historydata['close'].iloc[-2]>=point2):
                c= count_candles(historydata, historydata['high'].iloc[0],historydata['low'].iloc[0], historydata['high'].iloc[-2], historydata['low'].iloc[-2])
                if(c>2):
                    for i in lvl:
                        if(i>point2 and i<point2+60):
                            sl = i
                        else:
                            sl = point2+60

                    target = find_closest_level(point1,lvl,direction='downward') + 30
                    rrr = (abs(point2-target))/(sl-point2)
                    ready=1
                    print("\nCLOSING BHETLI !!! ON ",sigma)


        if(secitr == 0 and ready == 1 and rrr>=0.6 and count<1):
            if(c>2):                
                itr=itr+1
            if (c > 2 and 
            (point1 < historydata['close'].iloc[-1] <= (point1 + 10)) or 
            (point2 - 10) <= historydata['close'].iloc[-1] < point2):

                if(vrti==0):
                    entryprice = message['ltp']
                    if(today=="Wednesday" or tdate==monthly_exp_date):
                        sp =(int(math.floor(entryprice / 100.0)) * 100)-100
                    else:
                        sp =(int(math.floor(entryprice / 100.0)) * 100)

                    strike = expiry + str(sp) + "PE"
                    edata = {
                    "symbol": str(strike),
                    "qty":15,
                    "type":2,
                    "side":1,
                    "productType":"INTRADAY",
                    "limitPrice":0,
                    "stopPrice":0,
                    "validity":"DAY",
                    "disclosedQty":0,
                    "offlineOrder":False,
                    }
                    eresponse = fyers.place_order(data=edata)
                    print(eresponse)

                    print("ENTRY MADE FOR ",sigma,"ON ",entryprice)
                    entryhour = time.localtime(time.time()).tm_hour
                    entrymin =  time.localtime(time.time()).tm_min
                    entrydate = date.today()
                    count = count+1
                    status = 1
                    start_time = time.time()     
                else:
                    entryprice = message['ltp']
                    if(today=="Wednesday" or tdate==monthly_exp_date):
                        sp =(int(math.floor(entryprice / 100.0)) * 100)-100
                    else:
                        sp =(int(math.floor(entryprice / 100.0)) * 100)

                    strike = expiry + str(sp) + "CE"
                    edata = {
                    "symbol": str(strike),
                    "qty":15,
                    "type":2,
                    "side":1,
                    "productType":"INTRADAY",
                    "limitPrice":0,
                    "stopPrice":0,
                    "validity":"DAY",
                    "disclosedQty":0,
                    "offlineOrder":False,
                    }
                    eresponse = fyers.place_order(data=edata)
                    print(eresponse)

                    print("ENTRY MADE FOR ",sigma,"ON ",entryprice)
                    entryhour = time.localtime(time.time()).tm_hour
                    entrymin =  time.localtime(time.time()).tm_min
                    entrydate = date.today()
                    count = count+1
                    status = 1
                    start_time = time.time()   

        elif(target!=0 and sl!=0 and status==1 and ((message['ltp']>=target or message['ltp']<=sl) or pointchecker(message['ltp'],30)==True)):
            exit_pos = strike+"-INTRADAY"
            exdata = {"id":str(exit_pos)}
            exresponse = fyers.exit_positions(data=exdata)
            print(exresponse)
                
            
            exitprice = message['ltp']
            print("ENTRY EXITED FOR ",sigma,"ON ",exitprice)
            status = 0
            ready=0
            sl=0
            target=0


            with open('ic_backtest_report.csv', 'a', newline='') as csvfile:
                writer = csv.writer(csvfile)
                exitprice = message['ltp']
                exithour = time.localtime(time.time()).tm_hour
                exitmin =  time.localtime(time.time()).tm_min       #historydata.iloc[-1]['time']
                exitdate = date.today()
                pandl = entryprice-exitprice
                new_row = [symbols[0],entrydate,entryhour,entrymin,entryprice,exitdate,exithour,exitmin,exitprice,pandl]
                writer.writerow(new_row)
                print("exported trade data")

                if(message.get('symbol')==sym and count>=1):
                    data_type= "SymbolUpdate"
                    symbol_to_unsubscribe= [sym]
                    wfyers.unsubscribe(symbols= symbol_to_unsubscribe, data_type = data_type)
                        
        elif(status==1):
            current_time = time.time()
            elapsed_minutes = int((current_time - start_time) / 60)
            remaining_minutes = target_minutes - elapsed_minutes

            if(remaining_minutes <= 0 ):
                exit_pos = strike+"-INTRADAY"
                exdata = {"id":str(exit_pos)}
                exresponse = fyers.exit_positions(data=exdata)
                print(exresponse)


                exitprice = message['ltp']
                print("ENTRY EXITED FOR ",sigma,"ON ",exitprice)
                status = 0
                ready=0
                sl=0
                target=0



                with open('ic_backtest_report.csv', 'a', newline='') as csvfile:
                    writer = csv.writer(csvfile)
                    exitprice = message['ltp']
                    exithour = time.localtime(time.time()).tm_hour
                    exitmin =  time.localtime(time.time()).tm_min       #historydata.iloc[-1]['time']
                    exitdate = date.today()
                    pandl = entryprice-exitprice
                    new_row = [symbols[0],entrydate,entryhour,entrymin,entryprice,exitdate,exithour,exitmin,exitprice,pandl]
                    writer.writerow(new_row)
                    print("exported trade data")

                if(message.get('symbol')==sym and count>=1):
                    data_type= "SymbolUpdate"
                    symbol_to_unsubscribe= [sym]
                    wfyers.unsubscribe(symbols= symbol_to_unsubscribe, data_type = data_type)


    if(message.get('symbol')==sym and status!=0 and time.localtime(time.time()).tm_hour>=14 and time.localtime(time.time()).tm_min>=30):
        data_type= "SymbolUpdate"
        symbol_to_unsubscribe= [sym]
        wfyers.unsubscribe(symbols= symbol_to_unsubscribe, data_type = data_type)
        


def onerror(message):
    print("Error:", message)


def onclose(message):
    print("Connection closed:", message)


def onopen():
    data_type = "SymbolUpdate"

    symbols = [sym]
    wfyers.subscribe(symbols=symbols, data_type=data_type)
    
    wfyers.keep_running()


waccess_token = client_id+":"+access_token 

wfyers = data_ws.FyersDataSocket(
    access_token=waccess_token,       # Access token in the format "appid:accesstoken"
    log_path="",                     # Path to save logs. Leave empty to auto-create logs in the current directory.
    litemode=True,                  # Lite mode disabled. Set to True if you want a lite response.
    write_to_file=False,              # Save response in a log file instead of printing it.
    reconnect=True,                  # Enable auto-reconnection to WebSocket on disconnection.
    on_connect=onopen,               # Callback function to subscribe to data upon connection.
    on_close=onclose,                # Callback function to handle WebSocket connection close events.
    on_error=onerror,                # Callback function to handle WebSocket errors.
    on_message=onmessage            # Callback function to handle incoming messages from the WebSocket.
)

wfyers.connect()

{'type': 'cn', 'code': 200, 'message': 'Authentication done', 's': 'ok'} 	time:  5 54 40
{'type': 'lit', 'code': 200, 'message': 'Lite Mode On', 's': 'ok'} 	time:  5 54 41
{'type': 'sub', 'code': 200, 'message': 'Subscribed', 's': 'ok'} 	time:  5 54 42
{'ltp': 48159.0, 'symbol': 'NSE:NIFTYBANK-INDEX', 'type': 'if'} 	time:  5 54 43
